# Using Amazon Bedrock

Amazon Bedrock is a fully managed service that makes FMs from leading AI startups and Amazon available via an API, so you can choose from a wide range of FMs to find the model that is best suited for your use case.

This tutorial will show you how to use Amazon Bedrock endpoints and LangChain.

:::{Note}
this guide is for folks who are using the Amazon Bedrock endpoints. Check the [evaluation guide](../../getstarted/evaluation.md) if your using OpenAI endpoints.
:::

In [ ]:
# pip install ragas langchain pydantic pandas --upgrade

### Load sample dataset

In [ ]:
# data
from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
amnesty_qa

Now let us use the llm from Bedrock using `Bedrock` class from Langchain. 
Init a new instance of `Bedrock` with the `model_id` of the model you want to use. 
You will also have to init `BedrockEmbeddings` for evaluate function with the metrics that use embeddings (context_precision and context_recall).

In [ ]:
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

config = {
    "credentials_profile_name": "default",  # E.g "default"
    "region_name": "us-east-1",  # E.g. "us-east-1"
    "model_id": "cohere.command-text-v14",  # E.g "anthropic.claude-v2"
    "model_kwargs": {"temperature": 0.4},  # Your model parameters
    "embeddings_model": "cohere.embed-english-v3",  # your embeddings model
}

bedrock_model = Bedrock(
    credentials_profile_name=config["credentials_profile_name"],
    model_id=config["model_id"],
    model_kwargs=config["model_kwargs"],
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# init the embeddings
bedrock_embeddings = BedrockEmbeddings(
    credentials_profile_name=config["credentials_profile_name"],
    model_id=config["embeddings_model"],
)

Now we can use the llm and embeddings with `Bedrock` by passing it in the evaluate function.

### Evaluation

Running the evalutation is as simple as calling evaluate on the `Dataset` with the metrics of your choice.

First Let us import metrics that we are going to use

In [ ]:
from ragas.metrics import (
    context_precision,
    faithfulness,
    context_recall,
    answer_relevancy,
)
from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    context_precision,
    faithfulness,
    context_recall,
    answer_relevancy,
]

Let us wrap the bedrock LLM's as LangchainLLMWrapper 

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings.base import LangchainEmbeddingsWrapper

# Wrap the LLM's as ragas Langchain LLM's
ragas_critic_llm = LangchainLLMWrapper(bedrock_model)
# wrap the embeddings model
ragas_embeddings = LangchainEmbeddingsWrapper(bedrock_embeddings)

for m in metrics:
    # change LLM for metric
    m.__setattr__("llm", ragas_critic_llm)

    # check if this metric needs embeddings
    if hasattr(m, "embeddings"):
        # if so change the embeddings model
        m.__setattr__("embeddings", ragas_embeddings)

Now you call evaluate

In [ ]:
from ragas import evaluate

import nest_asyncio  # CHECK NOTES

# NOTES: Only used when running on a jupyter notebook, otherwise comment or remove this function.
nest_asyncio.apply()

result = evaluate(
    amnesty_qa["eval"].select(range(3)),
    metrics=metrics,
    llm=ragas_critic_llm,
    embeddings=ragas_embeddings,
)

result

and there you have the it, all the scores you need.

now if we want to dig into the results and figure out examples where your pipeline performed worse or really good you can easily convert it into a pandas array and use your standard analytics tools too!

In [ ]:
df = result.to_pandas()
df.head()

And thats it!

if you have any suggestion/feedbacks/things your not happy about, please do share it in the [issue section](https://github.com/explodinggradients/ragas/issues). We love hearing from you 😁